In [504]:
# imports
import numpy as np

In [505]:
# define the keys (e1, e2) and the s-boxes (s1, s2) here
e1 = np.array([1,0,0,1,1,0])
e2 = np.array([0,0,1,1,1,0])
s1 = np.array([[0,3,1,2],[2,3,0,1]])
s2 = np.array([[2,1,3,0],[0,1,2,3]])

In [506]:
# get the text from the user; output: an array of integers marking each character of the message
def getMessage():
  message = input("Enter your message: ")
  message = bytes(message, 'utf-8')
  message = np.array(bytearray(message))
  return message

In [507]:
# takes 4 bits and expands them to a 6-bit long sequence
def expand(bits):
  expanded = np.array([])
  expanded = np.append(expanded, bits[0])
  expanded = np.append(expanded, bits[3])
  expanded = np.append(expanded, bits[2])
  expanded = np.append(expanded, bits[1])
  expanded = np.append(expanded, bits[3])
  expanded = np.append(expanded, bits[0])
  return expanded

In [508]:
# XOR gate for bit sequences; output: a bitstring with the length of bits1 and bits2
def XOR(bits1, bits2):
  if(len(bits1)!=len(bits2)):
    print("ERROR: XOR: sequences are not the same length.")
    exit(1)
  else:
    result = np.array([])
    for x in range(len(bits1)):
      result = np.append(result, int(bits1[x]) ^ int(bits2[x]))
    return result

In [509]:
# SBoxes() takes 6 bits and processes them with the help of the s-boxes; output: 4 bits
def SBoxes(bits):
  firstHalf = bits[:3]
  secondHalf = bits[3:]
  result = ""

  result1 = bin(s1[int(firstHalf[0])][int(firstHalf[1]*2+firstHalf[2])])[2:].zfill(2)
  result2 = bin(s2[int(secondHalf[0])][int(secondHalf[1]*2+secondHalf[2])])[2:].zfill(2)

  result = result1 + result2

  return result

In [510]:
# verbose encryption routine
def encryptBits(bits):
  left = bits[:4]
  right = bits[4:]
  temp = np.array([]) # temp holds the temporary result
  bits_encrypted = "" # bits_encrypted holds the final result

  # Mini-DES algorithm, first round
  temp = expand(right)
  temp = XOR(temp, e1)
  temp = SBoxes(temp)
  temp = XOR(left, temp)
  left = right
  right = temp
  # second round
  temp = expand(right)
  temp = XOR(temp, e2)
  temp = SBoxes(temp)
  temp = XOR(left, temp)
  left = right
  right = temp

  # merge left and right half into a string (8-bit long)
  for b in left:
    bits_encrypted += str(int(b))
  for b in right:
    bits_encrypted += str(int(b))

  return bits_encrypted

In [ ]:
# msg_integers = getMessage()
msg_integers = np.array([ 84, 114, 101, 102, 102, 101, 110,  32, 119, 105, 114,  32, 117, 110, 115,  32, 109, 111, 114, 103, 101, 110,  63])

for integer in msg_integers:
  bits = bin(integer)       # convert integer to binary string
  bits = bits[2:].zfill(8)  # remove '0b' and pad the string with zeros on the left; bits is now 8-bit long
  print(bits)
  bits_encrypted = encryptBits(bits)  # bits_encrypted is a 8-bit long binary string as well
  print(bits_encrypted)
  print("----------------------------------------------------")

# for x in msg_integers:
#   print(x)
#   x = bin(x)          # convert integer to binary string
#   x = x[2:].zfill(8)  # remove '0b' and pad the string with zeros on the left; x is now 8-bit long
#   print(x)
#   print(int(x,2))     # convert binary string to integer
#   print(chr(int(x,2)))  # convert integer to character

# TODO:

- merge the output-octets into a long bitstring
- decrypt function which takes the long bitstring
- translate decrypted bitstring back into readable text